In [22]:
import aiohttp
import asyncio
import csv
import pandas as pd
from tqdm import tqdm
import json
import string
import math
import re
import emoji
import unicodedata as ud


tqdm.pandas()


In [23]:
df=pd.read_csv("out.csv")

In [24]:
df["tweet_len"]=df["tweet"].apply(lambda x: len(x))

In [19]:
# x=df[df["tweet_len"]==823]["tweet"].values

In [25]:
# x

In [26]:
def filter_text(text:str):
    flags={'U+1F1E6',
            'U+1F1E7',
            'U+1F1E9',
            'U+1F1EA',
            'U+1F1EC',
            'U+1F1ED',
            'U+1F1EE',
            'U+1F1EF',
            'U+1F1F0',
            'U+1F1F1',
            'U+1F1F2',
            'U+1F1F3',
            'U+1F1F4',
            'U+1F1F5',
            'U+1F1F6',
            'U+1F1F7',
            'U+1F1F8',
            'U+1F1F9',
            'U+1F1FC',
            'U+1F1FE',
            'U+1F1FF'}

    # remove emojis except for flags
    text  =''.join(char for char in text if (char in flags or not emoji.is_emoji(char)))
    # remove mentions
    text = re.sub("@[A-Za-z0-9_]+", "", text)
    # remove links
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    # remove hashes and keep the words
    text = text.replace("#", " ").replace("_", " ")
    # remove all english characters
    text = re.sub('[a-zA-Z0-9]+', '', text)
    # replace  multiple spaces with one space
    text = re.sub(' +', ' ',text)
    # remove all punctionations and all digits (arabic and english)
    text  =''.join(char for char in text if not (
                    ud.category(char).startswith('P') or 
                    ud.category(char).startswith('Nd')))
    return text
                   
   

In [28]:
df["clean"]=df["tweet"].progress_apply(filter_text)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 458197/458197 [01:14<00:00, 6121.36it/s]


In [29]:
df

,id,tweet,label,tweet_len,clean
0,1175358310087892992,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .,IQ,48,لكن بالنهاية ينتفض يغير
1,1175416117793349632,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...,IQ,120,يعني هذا محسوب على البشر حيونه ووحشيه وتطلب...
2,1175450108898565888,@KanaanRema مبين من كلامه خليجي,IQ,31,مبين من كلامه خليجي
3,1175471073770573824,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐,IQ,42,يسلملي مرورك وروحك الحلوه
4,1175496913145217024,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺,IQ,34,وين هل الغيبه اخ محمد
...,...,...,...,...,...
458192,1019484980282580992,@Al_mhbaa_7 مبسوطين منك اللي باسطانا😅,BH,37,مبسوطين منك اللي باسطانا
458193,1021083283709407232,@Zzainabali @P_ameerah والله ماينده ابش يختي,BH,44,والله ماينده ابش يختي
458194,1017477537889431552,@Al_mhbaa_7 شو عملنا لك حنا تهربي مننا احنا مس...,BH,73,شو عملنا لك حنا تهربي مننا احنا مساكين ليش بت...
458195,1022430374696239232,@haneenalmwla الله يبارك فيها وبالعافيه 😋😋😋,BH,43,الله يبارك فيها وبالعافيه


In [50]:

def remove_emoji(text):
    
    text  =''.join(char for char in text if  any ([not emoji.is_emoji(char),char in flags]))
                   
    return text


In [51]:
def remove_punct(text):
    
    text  =''.join(char for char in text if not (
        ud.category(char).startswith('P') or 
        ud.category(char).startswith('Nd')))
                   
    text = re.sub('[a-zA-Z0-9]+', '', text)
    return text


In [52]:

def remove_links(text):
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    return text


In [47]:
df["tweet_clean"] = df["tweet"].apply(remove_emoji)

df["tweet_clean"] = df["tweet_clean"].apply(lambda tweet: re.sub("@[A-Za-z0-9_]+", "", tweet))  # remove mentions

df["tweet_clean"] = df["tweet_clean"].apply(lambda tweet: re.sub(' +', ' ',tweet))


df["tweet_clean"] = df["tweet_clean"].apply(remove_links)


df["tweet_clean"] = df["tweet_clean"].apply(lambda tweet: tweet.replace("#", " ").replace("_", " "))  # split hashtags

df["tweet_clean"] = df["tweet_clean"].apply(remove_punct)

done
done
done
done
done


In [53]:
text="\n".join(df["tweet"])
with open("full tweets.txt","w") as f:
    f.write(text)

In [54]:
text="\n".join(df["tweet_clean"])

with open("full tweets clean.txt","w") as f:
    f.write(text)

In [ ]:
# 